In [1]:
# dependencies
!pip install gym box2d 

In [2]:
# import os 
# os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
import time
import gym

In [4]:
import DQN

In [5]:
dqn_agent = DQN.Agent(lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, batch_size=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               4608      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
Total params: 136,964
Trainable params: 136,964
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               4608      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_______________________

In [6]:
env = gym.make("LunarLander-v2")
spec = gym.spec("LunarLander-v2")
train = 1
test = 0
num_episodes = 5000
graph = True

file_type = 'h5'
file = 'saved_networks/dqn_model0'

In [7]:
t_start = time.perf_counter()

if train and not test:
    scores = dqn_agent.train_model(env, num_episodes, graph)
else:
    dqn_agent.test(env, num_episodes, file_type, file, graph)
    
t_end = time.perf_counter()

print(f"Finished in {t_end-t_start} seconds")

INFO:tensorflow:Assets written to: saved_networks/dqn_model0/assets
Network saved


KeyboardInterrupt: 

In [ ]:
np.savetxt('scores.out', np.array(scores), delimiter=',') # saving scores for each episode to scores.out